# **Data and Evidence CoP**
## Introduction to APIs

We will start by bringing in packages to our coding environment. These packages contain functions that allow us to do different things within our code.

We will be using:
- `requests`: a package for interacting with APIs
- `pandas`: given the short-form `pd`, is a package for working with dataframes, a type of data structure in Python that resembles a table
- `json`: data on the web is often stored in json format, so this package allows us to work with this data structure

In [9]:
import requests
import pandas as pd
import json

Now that we have installed all the functions that we need, we can start to define some variables that we need to access our data.

In [15]:
base_url = "http://environment.data.gov.uk/hydrology/id/stations"
print(base_url)

http://environment.data.gov.uk/hydrology/id/stations


If we visit that link, we can see it returns a lot of stations, 100 in fact. This is because that is the soft limit of the API, meaning that unless we specify a limit, or there are fewer than 1,000 items in the request, it will display the first 100 entries.
Let's set a limit so we only get the first ten entries.

Note that we have already defined the base url and the stations url, so we don't need to do this again.
To add a query to our URL, we first need to add a `?` and then we define our query. We put an `&` between each query if we have more than one.

In [ ]:
limit_query = "?_limit=10"
stations_10_url = base_url + limit_query
print(stations_10_url)

http://environment.data.gov.uk/hydrology/id/stations?_limit=10


Now that we're familiar with constructing a URL for accessing data, let's expand on this work by using the `requests` library to bring this data into our environment.
We use the `requests.get()` function to do this. 
This link provides more information on this method: https://www.w3schools.com/PYTHON/ref_requests_get.asp#:~:text=The%20get%28%29%20method%20sends%20a%20GET%20request%20to,timeout%3D2.50%29%20Required.%20The%20url%20of%20the%20request%20Optional.
In summary, the get() method sends a GET request. We provide it with a URL, and it accesses this URL, and simply "gets" some information from it.
Let's put our stations_10_url into the function. 

In [12]:
stations_10_response = requests.get(stations_10_url)
print(stations_10_response)

<Response [200]>


If we print our return from this function, it tells us that the status code of the return was 200. This means the request was successful.
Let's view the response.

In [19]:
if stations_10_response.status_code == 200:
    stations_10_json = stations_10_response.json()
    print(stations_10_json)
else:
    print(f"Error: {stations_10_response.status_code}")

{'meta': {'@id': 'http://environment.data.gov.uk/hydrology/id/stations?_limit=10', 'publisher': 'Environment Agency', 'license': 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/', 'licenseName': 'OGL 3', 'comment': 'Hydrology API for sub-daily data', 'version': '2.0.0', 'hasFormat': ['http://environment.data.gov.uk/hydrology/id/stations.csv?_limit=10', 'http://environment.data.gov.uk/hydrology/id/stations.ttl?_limit=10', 'http://environment.data.gov.uk/hydrology/id/stations.json?_limit=10', 'http://environment.data.gov.uk/hydrology/id/stations.html?_limit=10', 'http://environment.data.gov.uk/hydrology/id/stations.rdf?_limit=10', 'http://environment.data.gov.uk/hydrology/id/stations.geojson?_limit=10'], 'limit': 10}, 'items': [{'@id': 'http://environment.data.gov.uk/hydrology/id/stations/052d0819-2a32-47df-9b99-c243c9c8235b', 'label': 'Ulting Sarasota', 'notation': '052d0819-2a32-47df-9b99-c243c9c8235b', 'easting': 581271, 'northing': 208591, 'lat': 51.746683, 

This isn't the neatest looking thing to print. But we can see that the JSON is made of two main elements, the `meta` and the `items`.
Meta contains metadata on things such as publisher and licenses.
In our case, we are interested in the items section, as this contains information on each station.
Let's look into the items in more detail.

In [20]:
stations = stations_10_json['items']
stations[0]

{'@id': 'http://environment.data.gov.uk/hydrology/id/stations/052d0819-2a32-47df-9b99-c243c9c8235b',
 'label': 'Ulting Sarasota',
 'notation': '052d0819-2a32-47df-9b99-c243c9c8235b',
 'easting': 581271,
 'northing': 208591,
 'lat': 51.746683,
 'long': 0.624437,
 'type': [{'@id': 'http://environment.data.gov.uk/flood-monitoring/def/core/Station'},
  {'@id': 'http://environment.data.gov.uk/flood-monitoring/def/core/RiverStation'},
  {'@id': 'http://environment.data.gov.uk/reference/def/core/SamplingLocation'},
  {'@id': 'http://environment.data.gov.uk/flood-monitoring/def/core/RiverFlow'}],
 'riverName': 'River Chelmer',
 'stationGuid': '052d0819-2a32-47df-9b99-c243c9c8235b',
 'wiskiID': '037048U',
 'dateOpened': '2008-10-31',
 'observedProperty': [{'@id': 'http://environment.data.gov.uk/reference/def/op/waterFlow'}],
 'status': [{'@id': 'http://environment.data.gov.uk/flood-monitoring/def/core/statusActive',
   'label': 'Active'}],
 'measures': [{'@id': 'http://environment.data.gov.uk/h

We have successfully pulled in data from an API into our environment in JSON format.
After a while you get used to reading data in JSON format, but it isn't the nicest thing to view. Let's convert it to a DataFrame.
A DataFrame is a way of displaying data in a table format. It is much easier to look at and work with.

In [21]:
stations_df = pd.DataFrame(stations)
stations_df

,@id,label,notation,easting,northing,lat,long,type,riverName,stationGuid,...,observedProperty,status,measures,stationReference,RLOIid,rloiStationLink,catchmentArea,nrfaStationID,nrfaStationURL,statusReason
0,http://environment.data.gov.uk/hydrology/id/st...,Ulting Sarasota,052d0819-2a32-47df-9b99-c243c9c8235b,581271,208591,51.746683,0.624437,[{'@id': 'http://environment.data.gov.uk/flood...,River Chelmer,052d0819-2a32-47df-9b99-c243c9c8235b,...,[{'@id': 'http://environment.data.gov.uk/refer...,[{'@id': 'http://environment.data.gov.uk/flood...,[{'@id': 'http://environment.data.gov.uk/hydro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://environment.data.gov.uk/hydrology/id/st...,Beggearn Huish,48513a18-e485-4317-ae92-93bf4f7f3e54,304007,139460,51.146322,-3.373695,[{'@id': 'http://environment.data.gov.uk/flood...,Washford River,48513a18-e485-4317-ae92-93bf4f7f3e54,...,[{'@id': 'http://environment.data.gov.uk/refer...,[{'@id': 'http://environment.data.gov.uk/flood...,[{'@id': 'http://environment.data.gov.uk/hydro...,51107,3089,{'@id': 'https://check-for-flooding.service.go...,36.3,51003,https://nrfa.ceh.ac.uk/data/station/info/51003...,NaN
2,http://environment.data.gov.uk/hydrology/id/st...,Adwick,f22f80f8-1bb0-4e77-b225-291487060c6f,447677,402022,53.512713,-1.282505,[{'@id': 'http://environment.data.gov.uk/flood...,River Dearne,f22f80f8-1bb0-4e77-b225-291487060c6f,...,[{'@id': 'http://environment.data.gov.uk/refer...,[{'@id': 'http://environment.data.gov.uk/flood...,[{'@id': 'http://environment.data.gov.uk/hydro...,F0803,8002,{'@id': 'https://check-for-flooding.service.go...,311.0,27030,https://nrfa.ceh.ac.uk/data/station/info/27030...,NaN
3,http://environment.data.gov.uk/hydrology/id/st...,Wellesbourne,95a1245a-5329-4a2b-9024-d70ebe9707df,427271,255587,52.197839,-1.602416,[{'@id': 'http://environment.data.gov.uk/flood...,River Dene,95a1245a-5329-4a2b-9024-d70ebe9707df,...,[{'@id': 'http://environment.data.gov.uk/refer...,[{'@id': 'http://environment.data.gov.uk/flood...,[{'@id': 'http://environment.data.gov.uk/hydro...,2048,2029,{'@id': 'https://check-for-flooding.service.go...,NaN,54048,https://nrfa.ceh.ac.uk/data/station/info/54048...,NaN
4,http://environment.data.gov.uk/hydrology/id/st...,Thorverton,3c4d4f78-2d0e-474a-b884-65a9daca18fb,293602,101602,50.804172,-3.511302,[{'@id': 'http://environment.data.gov.uk/flood...,River Exe,3c4d4f78-2d0e-474a-b884-65a9daca18fb,...,[{'@id': 'http://environment.data.gov.uk/refer...,[{'@id': 'http://environment.data.gov.uk/flood...,[{'@id': 'http://environment.data.gov.uk/hydro...,45118,3260,{'@id': 'https://check-for-flooding.service.go...,600.9,45001,https://nrfa.ceh.ac.uk/data/station/info/45001...,NaN
5,http://environment.data.gov.uk/hydrology/id/st...,Iwood,959f3e4f-bb6e-4f4a-8082-0157eea99482,345173,163064,51.363977,-2.788890,[{'@id': 'http://environment.data.gov.uk/flood...,Congresbury Yeo,959f3e4f-bb6e-4f4a-8082-0157eea99482,...,[{'@id': 'http://environment.data.gov.uk/refer...,[{'@id': 'http://environment.data.gov.uk/flood...,[{'@id': 'http://environment.data.gov.uk/hydro...,52204,"[3060, 3416]",[{'@id': 'https://check-for-flooding.service.g...,NaN,52017,https://nrfa.ceh.ac.uk/data/station/info/52017...,NaN
6,http://environment.data.gov.uk/hydrology/id/st...,Coniston Upstream,fbecd84a-70b6-4725-80ba-67c7adb3cae7,330639,497190,54.365471,-3.069004,[{'@id': 'http://environment.data.gov.uk/flood...,Church Beck,fbecd84a-70b6-4725-80ba-67c7adb3cae7,...,[{'@id': 'http://environment.data.gov.uk/refer...,[{'@id': 'http://environment.data.gov.uk/flood...,[{'@id': 'http://environment.data.gov.uk/hydro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,http://environment.data.gov.uk/hydrology/id/st...,Sheepwash,70f658a0-3d32-47db-a97c-ed5e25da46f2,397397,291756,52.523665,-2.039788,[{'@id': 'http://environment.data.gov.uk/flood...,River Tame,70f658a0-3d32-47db-a97c-ed5e25da46f2,...,[{'@id': 'http://environment.data.gov.uk/refer...,[{'@id': 'http://environment.data.gov.uk/flood...,[{'@id': 'http://environment.data.gov.uk

We have successfully pulled in data using an API. 
This isn't the most interesting dataset though. Let's access some slightly more interesting data and use a more complex query than just a limit.
We can define the `parameters` of our query and use them in our request. This is a nicer way of writing a request than making a longer and longer URL.